Eithar Elfatih Burie Abdelrahman Batch DS2403

In [144]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
from selenium.common.exceptions import NoSuchElementException
import re

__________________________

1.Write a python program which searches all the product under a particular product from www.amazon.in. The product to be searched will be taken as input from user. 

2. In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a data frame and csv. In case if any product has less than 3 pages in search results then scrape all the products available under that product name. Details to be scraped are: "Brand Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“. 

Enter required product:men's trousers
No more pages to load.


In [191]:
def scrape_amazon_products(search_input, max_pages=3):
    amazon_url = "https://www.amazon.in"
    driver = webdriver.Chrome()  
    driver.get(amazon_url)
# Call the search function
    search_amazon(driver, search_input)  

    product_data = []
    for page_num in range(1, max_pages + 1):
        product_elements = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.XPATH, '//div[@data-component-type="s-search-result"]'))
        )
        for element in product_elements:
            product_url = element.find_element(By.XPATH, './/a[@class="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"]').get_attribute('href')
            
            try:
                brand_name = element.find_element(By.XPATH, './/span[@class="a-size-base-plus a-color-base a-text-normal"]').text
            except:
                brand_name = "-"
            try:
                product_name = element.find_element(By.XPATH, './/span[@class="a-size-base-plus a-color-base a-text-normal"]').text
            except:
                product_name = "-"
            try:
                product_price = element.find_element(By.XPATH, './/span[@class="a-offscreen"]').text
            except:
                product_price = "-"
            try:
                return_exchange = element.find_element(By.XPATH, './/div[contains(@aria-label, "FREE Delivery")]/following-sibling::div[1]').text
            except:
                return_exchange = "-"
            try:
                expected_delivery = element.find_element(By.XPATH, './/span[@class="a-text-bold"]').text
            except:
                expected_delivery = "-"
            try:
                availability = element.find_element(By.XPATH, './/span[contains(text(), "In stock") or contains(text(), "Usually dispatched in")]').text
            except:
                availability = "-"

            product_data.append({
                'Brand Name': brand_name,
                'Product Name': product_name,
                'Price': product_price,
                'Return/Exchange': return_exchange,
                'Expected Delivery': expected_delivery,
                'Availability': availability,
                'Product URL': product_url
            })

        # Check for next page and click if available
        try:
            next_page_button = driver.find_element(By.XPATH, '//a[@class="s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"]')
            next_page_button.click()
            time.sleep(2) 
        except:
            #break the loop
            break

    driver.quit()  # Close the browser
    df = pd.DataFrame(product_data)
    df.to_csv("amazon_products.csv", index=False)  # Save to CSV
    return df


def search_amazon(driver, input_search):
    #Use presence_of_element_located
    search_box = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "twotabsearchtextbox"))
    )
    search_box.send_keys(input_search)
    search_box.submit()


# Get the search term from the user
search_term = input("Enter the product to search for: ")

# Perform the search and scrape the data
product_data_df = scrape_amazon_products(input_search)
product_data_df.index=product_data_df.index+1
product_data_df


Enter the product to search for: men's trousers


,Brand Name,Product Name,Price,Return/Exchange,Expected Delivery,Availability,Product URL
1,Men Solid Casual Regular Fit Trousers,Men Solid Casual Regular Fit Trousers,,-,-,-,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
2,Men's Solid Formal Trousers,Men's Solid Formal Trousers,,-,-,-,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
3,Mens Cotton Lycra Mid Rise Straight Solid Chinos,Mens Cotton Lycra Mid Rise Straight Solid Chinos,,-,-,-,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
4,Men's Slim Fit Cargos,Men's Slim Fit Cargos,,-,-,-,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
5,Men Trouser || Men's Regular Trouser || Men's ...,Men Trouser || Men's Regular Trouser || Men's ...,,-,-,-,https://www.amazon.in/Trouser-Regular-Casual-T...
...,...,...,...,...,...,...,...
176,Men's Solid Tailored Fit Polyester Formal Trouser,Men's Solid Tailored Fit Polyester Formal Trouser,,-,-,-,https://www.amazon.in/Arrow-Regular-Pants-ARAD...
177,Men Cargos,Men Cargos,,-,-,-,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
178,Men's Cotton Printed Camoflague Cargo Trouser,Men's Cotton Printed Camoflague Cargo Trouser,,-,-,-,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
179,Men's Slim Fit Stretch Trousers (Business Casu...,Men's Slim Fit Stretch Trousers (Business Casu...,,-,-,-,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...


________

3.Write a python program to access the search bar and search button on images.google.com and scrape 10 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’.  

In [116]:
# Create a new chrome driver
driver = webdriver.Chrome()

#create keywords list
keywords = ['fruits', 'cars', 'Machine Learning', 'Guitar', 'Cakes']

for keyword in keywords:
    # Open images.google.com
    driver.get('https://images.google.com')

    #find search bar. enter keywords
    search_=driver.find_element(By.XPATH, "//textarea[@class='gLFyf']")
    search_.send_keys(keyword)
    
    button_=driver.find_element(By.XPATH, "//button[@class='Tg7LZd']")
    button_.click()
    
    #scroll to load more images
    for _ in range(3):
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(5)
    
    #find image details,extract urls 
    image_dets=driver.find_elements(By.TAG_NAME, 'img')
    image_urls=[element.get_attribute('src') for element in image_dets]
    image_urls=[url for url in image_urls if url and url.startswith('https')]
    
    #print first ten
    
    print(f'\nTop 10 images for "{keyword}" :')
    for i in range(min(10, len(image_urls))):
        print(f'{i+1}. {image_urls[i]}')

#close driver           
driver.quit()
   


Top 10 images for "fruits" :
1. https://www.google.com/images/branding/googlelogo/2x/googlelogo_color_92x30dp.png
2. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSfXdjbHBJX06KUKN4tkfd8KFFDyYnIICCvDxYxgY3_0Plx7p71-aCeB-fp&s
3. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQhRe8IDwbhaqr66uyVbS3Mcn8qLL5bqhVNrKr19LhFvDtTE3wmthx8m3Zx&s
4. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQJI-r9JDFn4sJnHXt76kL8lBBybGhtbRz3ArtMtSSGdocJlaSoA2sZPze4&s
5. https://fonts.gstatic.com/s/i/productlogos/googleg/v6/24px.svg
6. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRb8pOojAEhqtRfCks1g-qoZQStlPtMbDrJ-Q&s
7. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQXPbFb9nYxZTmrHzr3SpbS4fYQlq1rDLDhAg&s
8. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSI-UzsUzLpfTKFBJPysBoJDQqiS1w1SYDzeg&s
9. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQiTNyEzEUByF7Z2Hp8CmS8Va721UxKICTpEQ&s
10. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQQ8ZX1beBSP-ydVjHGJdSiH

__________________________________________

4.Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”,  
“Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.  

In [192]:
# open driver
driver=webdriver.Chrome()
driver.get('https://www.flipkart.com')

# enter keyword into sesrch bar and click 
designation=driver.find_element(By.XPATH, "//input[@class='Pke_EE']")
designation.send_keys("Samsung Galaxy S23")

#
search=driver.find_element(By.XPATH, "//button[@class='_2iLD__']")
search.click()

In [201]:
SmartPhoneNameAndColour=[]
RAM=[]
StorageROM=[]
PrimeCam=[]
SecondCam=[]
DisplaySize=[]
BatteryCapacity=[]
Price=[]
ProductURL=[]


In [202]:
# scrap SmartPhoneNameAndColour
name_tags=driver.find_elements(By.XPATH, "//div[@class='KzDlHZ']")
for i in name_tags:
    name=i.text
    SmartPhoneNameAndColour.append(name)
    
# scrap RAM
RAM_tags=driver.find_elements(By.XPATH, ".//ul[@class='G4BRas']")
for i in RAM_tags:
    ram=i.text
    RAM.append(ram)
    


In [203]:
#check length of all details are equal for Dataframe to be created 
print(len(SmartPhoneNameAndColour),len(RAM))

24 24


________________________________

5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps. 

In [55]:
#open goole maps on automated chrome
driver=webdriver.Chrome()
driver.get("https://www.google.com/maps")
time.sleep(5)

city=input('Enter city: ')

designation=driver.find_element(By.ID, "searchboxinput")
designation.send_keys(city)
time.sleep(5)

search=driver.find_element(By.ID, "searchbox-searchbutton")
search.click()
time.sleep(5)


try:
    url_=driver.current_url
    print("Url Extracted:", url_)
    lat_long=re.findall(r'@(.*)data',url_)
    if len(lat_long):
        lat_long_= lat_long[0].split(",")
        if len(lat_long_)>=2:
            lat=lat_long_[0]
            long=lat_long_[1]
        print("Latitude = {}, Longitude= {}".format(lat, long))
        
except Exception as e:
    print("Error", str(e))

Enter city: Nasr City 
Url Extracted: https://www.google.com/maps/place/Nasr+City,+Al+Manteqah+Al+Oula,+Nasr+City,+Cairo+Governorate+4450113/@30.1254945,31.3198379,15z/data=!4m6!3m5!1s0x14583e5d94c66301:0xddddf100de42206c!8m2!3d30.0566104!4d31.3301076!16s%2Fg%2F11xcyk3j2?entry=ttu
Latitude = 30.1254945, Longitude= 31.3198379


__________

6.Write a program to scrap all the available details of best gaming laptops from digit.in.  

In [ ]:
driver=webdriver.Chrome()

In [ ]:
driver.get('https://www.digit.in.')

In [ ]:
designation=driver.find_element(By.XPATH, "/html/body/div[1]/header/div/div[1]/div/div/div[2]/form/input[1]")
designation.send_keys("gaming laptops")

________________________________

7.Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.  

In [5]:
#open chrome 
driver=webdriver.Chrome()
#load required webpage
driver.get('https://www.forbes.com/billionaires/')

time.sleep(5)


In [6]:
Rank=[]
Name=[]
NetWorth=[]
Age=[]
CountryTerritory=[]
Source=[]
Industry=[]


In [8]:
ranks=driver.find_element(By.XPATH, "//div[@class='Table_rank__X4MKf']/div")
for i in ranks:
    Rank.append(i.text)
    
timesleep(2)

names=driver.find_element(By.XPATH, "//div[@class='Table_personName__Bus2E']")
for i in names:
    Name.append(i.text)
    
timesleep(2)

net_=driver.find_element(By.XPATH, "//div[@class='Table_finalWorth__UZA6k']")
for i in net_:
    NetWorth.append(i.text)
    
timesleep(2)

age=driver.find_element(By.XPATH, "//div[@role='cell']//span")
for i in age:
    Age.append(i.text)
    
timesleep(2)

country=driver.find_element(By.XPATH, "")
for i in country:
    CountryTerritory.append(i.text)
    
timesleep(2)

sources=driver.find_element(By.XPATH, "//div[@class='Table_columnHeader__Js_Vi']")
for i in sources:
    Source.append(i.text)
    
timesleep(2)

industry=driver.find_element(By.XPATH, "")
for i in industry:
    Industry.append(i.text)
    
timesleep(2)

TypeError: 'WebElement' object is not iterable

________

8.Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted 
from any YouTube Video.

In [119]:
#open chrome webdriver
driver=webdriver.Chrome()
driver.get('https://www.youtube.com/')




In [120]:
#finding element for search bar
search_bar = driver.find_element(By.XPATH, '/html/body/ytd-app/div[1]/div/ytd-masthead/div[4]/div[2]/ytd-searchbox/form/div[1]/div[1]/input')
search_bar.send_keys("beyonce - crazy in love") 
time.sleep(1)

#clicking on search button
search_btn = driver.find_element(By.ID, "search-icon-legacy")  
search_btn.click()
time.sleep(4)

#clicking on first video
link_click = driver.find_element(By.XPATH, "/html/body/ytd-app/div[1]/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[2]/ytd-item-section-renderer/div[3]/ytd-video-renderer[2]/div[1]/div/div[1]/div/h3/a")
link_click.click()

In [122]:
#extract comments, upvotes and time
comment=[]
comment_date=[]
up_vote=[]

In [124]:
#scroll to comments
scroll_time= 2 #seconds
scrolls= 15

for i in range(scrolls):
    driver.execute_script('window.scrollTo(0,document.documentElement.scrollHeight);')
    time.sleep(scroll_time)
    


    comments=driver.find_elements(By.XPATH, "//div[@class='style-scope ytd-expander']/yt-attributed-string")
    for i in comments:
        if i.text is None:
            comment.append('--')
        else:
            comment.append(i.text)
    time.sleep(2)

    upvotes=driver.find_elements(By.XPATH, "//span[@class='style-scope ytd-comment-engagement-bar']")
    for i in upvotes:
        up_vote.append(i.text)
        
    time.sleep(2)
    
    times=driver.find_elements(By.XPATH, "//span[@id='published-time-text']")
    for i in times:
        comment_date.append(i.text)
    
    for i in range(0,len(comment_date),2):
        comment_date.append(comment_date[i])
        
    time.sleep(2)



In [125]:
print(len(comments), len(upvotes), len(times))

0 0 0


In [110]:
df_youtube=pd.DataFrame({'Comment':comment[0:500],
                'Comment Time': comment_date[0:500],
                'Up vote': up_vote[0:500]})
df_youtube.index=df_youtube.index +1
df_youtube

,Comment,Comment Time,Up vote


In [ ]:
driver.close()

______________

9.Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in “London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, privates from price, dorms from price, facilities and property description.  

In [ ]:
#open driver
driver=webdriver.Chrome()
driver.get('https://www.hostelworld.com/')

#find and enter 'london for location
location=driver.find_element(By.XPATH, "//input[@class='native-input']")
location.send_keys('London')
time.sleep(2)

#
location2=driver.find_element(By.XPATH, "/html/body/div[3]/div/div[2]/main/header/div/div[2]/div[1]/div[1]/div/div[1]/div[2]/div/ul/li[2]/button")
location2.click()
time.sleep(2)

#find and click search
search=driver.find_element(By.XPATH, "/html/body/div[3]/div/div[2]/main/header/div/div[2]/div[1]/div[1]/div/div[5]/button[1]")
search.click()


In [ ]:
#all hostel containers 
hostel=driver.find_elements(By.XPATH, "//div[@class='property-card']"

for hostel in hostels:
    try:
        name=

In [ ]:
#create lists
hostel_name=[]
distance_=[]
ratings_=[]
tot_reviews=[]
overall_review=[]
private_price=[]
dorm_price=[]
facilities_=[]
prop_description=[]

In [ ]:
#scraping hostel_name from given page
name_tags=driver.find_elements(By.XPATH, "//div[@class='property-name']/span") 
for i in name_tags:
    name=i.text
    hostel_name.append(name)
   
#scraping distance from given page
distance_tags=driver.find_elements(By.XPATH, "//span[@class='distance-description']") 
for i in distance_tags:
    distance=i.text
    distance_.append(distance)

#scraping ratings from given page
ratings_tags=driver.find_elements(By.XPATH, "//div[@class='rating medium theme-null']") 
for i in ratings_tags:
    rating=i.text
    ratings_.append(ratings)
    
#scraping ratings from given page
total_tags=driver.find_elements(By.XPATH, "//div[@class='rating medium theme-null']") 
for i in total_tags:
    total=i.text
    tot_review.append(total)